In [2]:
import cv2
import numpy as np #for arrays

#Used for confidence (Only shows higher than the value below)
THRESHOLD=0.2

#Target size for the input
TARGET_SIZE = 700

#Declaring the FONT
FONT = cv2.FONT_HERSHEY_SIMPLEX

#Import model weights from file
protoFile = "Model/pose_deploy_linevec.prototxt"
weightsFile = "Model/pose_iter_440000.caffemodel"
POSE_MODEL = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
#POSE_MODEL = cv2.dnn.readNetFromTensorflow("Model/graph_opt.pb") #Mobile Version runs smoother but less accurate

print(cv2.cuda.getDevice())

#To use GPU instead of CPU
POSE_MODEL.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
POSE_MODEL.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

0


In [3]:
def PoseEstimationFromMedia(PATH_STRING):
    #Import image to process
    MEDIA_RAW = cv2.VideoCapture(PATH_STRING if PATH_STRING else 0)

    while cv2.waitKey(1) < 0:

        hasFrame, frame = MEDIA_RAW.read()

        if not hasFrame:
            cv2.waitKey()
            cv2.destroyAllWindows()
            break

        frame = cv2.resize(frame, (int((TARGET_SIZE/frame.shape[0])*frame.shape[1]),TARGET_SIZE))

        IMG_HEIGHT = frame.shape[0]
        IMG_WIDTH = frame.shape[1]

        #Resize for prediction
        BLOB_HEIGHT=368
        BLOB_WIDTH=int((BLOB_HEIGHT/IMG_HEIGHT)*IMG_WIDTH)

        # Use the given image as input, which needs to be blob(s).
        imgBlob = cv2.dnn.blobFromImage(frame, 1.0/255, (BLOB_WIDTH, BLOB_HEIGHT), (0,0,0), swapRB=True, crop=False)
        #imgBlob = cv2.dnn.blobFromImage(frame, 1.0, (BLOB_WIDTH, BLOB_HEIGHT), (127.5, 127.5, 127.5), swapRB=True, crop=False) #with mobile
        POSE_MODEL.setInput(imgBlob)

        # Runs a forward pass to compute the POSE_MODEL output
        out = POSE_MODEL.forward()
        # MobilePOSE_MODEL output [1, 57, -1, -1], we only need the first 19 elements
        out = out[:, :19, :, :]

        assert(len(BODY_PARTS) == out.shape[1])

        points = []
        for i in range(len(BODY_PARTS)):
            # Slice heatmap of corresponding body's part.
            heatMap = out[0, i, :, :]

            # Originally, we try to find all the local maximums. To simplify a sample
            # we just find a global one. However only a single pose at the same time
            # could be detected this way.
            _, conf, _, point = cv2.minMaxLoc(heatMap)
            x = (IMG_WIDTH * point[0]) / out.shape[3]
            y = (IMG_HEIGHT * point[1]) / out.shape[2]
            # Add a point if it's confidence is higher than THRESHOLD.
            points.append((int(x), int(y)) if conf > THRESHOLD else None)


        for pair in POSE_PAIRS:
            partFrom = pair[0]
            partTo = pair[1]
            assert(partFrom in BODY_PARTS)
            assert(partTo in BODY_PARTS)

            idFrom = BODY_PARTS[partFrom]
            idTo = BODY_PARTS[partTo]

            if points[idFrom] and points[idTo]:
                cv2.putText(frame,partFrom,points[idFrom], FONT, 0.5, (255,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame,partTo,points[idTo],FONT, 0.5, (255,0,0), 1, cv2.LINE_AA)
                cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
                cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
                cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)


        #For .ms in top-left corner
        t, _ = POSE_MODEL.getPerfProfile()
        freq = cv2.getTickFrequency() / 1000
        cv2.putText(frame, '%.2fms' % (t / freq), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

        #Show Modified image
        cv2.imshow("Pose Estimation",frame)

    cv2.destroyAllWindows()


#Reading an Image
#PoseEstimationFromMedia("Media/Front_Male.jpg")
#PoseEstimationFromMedia("Media/Side_Female.jpg")
#PoseEstimationFromMedia("Media/Front_Female.jpg")
#PoseEstimationFromMedia("Media/Male_Pose.jpg")
#PoseEstimationFromMedia("Media/Side_FemaleAndMale.jpg")
PoseEstimationFromMedia(0)